<h1>Run SEIRHUD</h1>

In [1]:
from model import SEIRHUD
import csv
import numpy as np
import pandas as pd
import time
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [7]:
data = pd.read_csv("../data/salvador.csv")
data.head()

,date,city,cases,deaths,ndeaths
0,2020-03-13,Salvador,3,0,0
1,2020-03-14,Salvador,3,0,0
2,2020-03-15,Salvador,4,0,0
3,2020-03-16,Salvador,4,0,0
4,2020-03-17,Salvador,5,0,0


In [4]:
def bootWeig(series, times):
    series = np.diff(series)
    series = np.insert(series, 0, 1)
    results = []
    for i in range(0,times):
        results.append(np.random.multinomial(n = sum(series), pvals = series/sum(series)))
    return np.array(results)

In [8]:
#using bootstrap to 
infeclists = bootWeig(data["cases"], 500)
deathslists = bootWeig(data["deaths"], 500)

In [9]:
#Define empty lists to recive results
ypred = []
dpred = []
spred = []
epred = []
beta1 = []
beta2 = []
gammaH = []
gammaU = []
delta = []
ia0 = []
t1 = []
e0 = []
is0 = []

In [10]:
#define fixed parameters:
kappa = 1/4
p = 0.2
gammaA = 1/3.5
gammaS = 1/4
muH = 0.15
muU = 0.4
xi = 0.53
omega_U = 0.29 
omega_H = 0.14
N = 2872347

#Bound   b1,   b2?, t1, gmH  gmU    d   h     ia0   is0  e0
bound = ([0,0,0,1/14,1/14,0,0.05,0,0,0],
         [30,1.5,1,1/5,1/5,1,0.35,10/N,10/N,10/N])

In [11]:
for cases, deaths in tqdm(zip(infeclists, deathslists)):
    model = SEIRHUD(tamanhoPop = N, numeroProcessadores = 8)
    model.fit(x = range(1,len(data["cases"]) + 1),
          y = np.cumsum(cases),
          d = np.cumsum(deaths),
          bound = bound,
          kappa = kappa,
          p = p,
          gammaA = gammaA,
          gammaS = gammaS,
          muH = muH,
          muU = muU,
          xi = xi,
          omegaU = omega_U,
          omegaH = omega_H,
          stand_error = True,
          )
    results = model.predict(range(1,len(data["cases"]) + 200))
    
    coef = model.getCoef()
    
    #Append predictions
    ypred.append(results["pred"])
    dpred.append(results["death"])
    spred.append(results["susceptible"])
    epred.append(results["exposed"])
    
    
    #append parameters
    beta1.append(coef["beta1"])
    beta2.append(coef["beta2"])
    gammaH.append(coef["gammaH"])
    gammaU.append(coef["gammaU"])
    delta.append(coef["delta"])
    t1.append(coef["dia_mudanca"])
    ia0.append(coef["ia0"])
    e0.append(coef["e0"])
    is0.append(coef["is0"])

0it [00:00, ?it/s]2020-06-29 18:26:05,797 - pyswarms.single.local_best - INFO - Optimize for 250 iters with {'c1': 0.3, 'c2': 0.3, 'w': 0.9, 'k': 5, 'p': 2}

pyswarms.single.local_best:   0%|          |0/250


TypeError: objectiveFunction() got an unexpected keyword argument 'verbose'

In [ ]:
def getConfidenceInterval(series, length):
    series = np.array(series)
    
    #Compute mean value
    meanValue = [np.mean(series[:,i]) for i in range(0,length)]

    #Compute deltaStar
    deltaStar = meanValue - series

    #Compute lower and uper bound
    deltaL = [np.quantile(deltaStar[:,i], q = 0.025) for i in range(0,length)]
    deltaU = [np.quantile(deltaStar[:,i], q = 0.975) for i in range(0,length)]

    #Compute CI
    lowerBound  = np.array(meanValue) + np.array(deltaL)
    UpperBound  = np.array(meanValue) + np.array(deltaU)
    return [meanValue, lowerBound, UpperBound]

In [ ]:
#Get confidence interval for prediction
for i, pred in tqdm(zip([ypred, dpred, epred, spred],
                   ["Infec", "deaths", "exposed", "susceptible"])):
    Meanvalue, lowerBound, UpperBound = getConfidenceInterval(i, len(data["cases"]) + 199)
    df = pd.DataFrame.from_dict({pred + "_mean": Meanvalue,
                            pred + "_lb": lowerBound,
                            pred + "_ub": UpperBound})
    df.to_csv("../results/Salvador/" + pred + ".csv", index = False)

In [ ]:
#Exprort parametes
parameters = pd.DataFrame.from_dict({"beta1": beta1,
                                     "beta2": beta2,
                                     "gammaH": gammaH,
                                     "gammaU": gammaU,
                                     "delta": delta,
                                     "ia0":ia0,
                                     "e0": e0,
                                     "t1": t1,
                                     "is0":is0})
parameters.to_csv("../results/Salvador/Parameters.csv", index = False)